In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
import random
from urllib.parse import urljoin
import json

base_url = "https://pt.wikipedia.org"
paginas_visitadas = set()
fila_paginas = [base_url]
total_paginas = 0
max_paginas = 5000
diretorio_dados = "paginas_wikipedia"
diretorio_infoboxes = "infoboxes_wikipedia"
tempo_espera = 1

if not os.path.exists(diretorio_dados):
    os.makedirs(diretorio_dados)

mapa_titulos = {}

def normalizar_nome_arquivo(titulo):
    caracteres_invalidos = ['\\', '/', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in caracteres_invalidos:
        titulo = titulo.replace(char, ' ')
    
    while '  ' in titulo:
        titulo = titulo.replace('  ', ' ')
    
    titulo = titulo.strip()
    
    if len(titulo) > 100:
        titulo = titulo[:100]
    
    return titulo

def obter_pagina(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Erro ao acessar: {url}. Código: {response.status_code}")
            return None
    except Exception as e:
        print(f"Erro ao acessar: {url}. Erro: {str(e)}")
        return None

def processar_links(soup):
    links = []
    
    todos_links = soup.find_all("a")
    for link in todos_links:
        if "href" in link.attrs.keys() and link["href"].startswith("/wiki/"):
            if ":" in link["href"]:
                continue
            
            url_completa = urljoin(base_url, link["href"])
            
            if url_completa not in paginas_visitadas and url_completa not in fila_paginas:
                links.append(url_completa)
    
    return links

def salvar_pagina(conteudo, soup):
    try:
        titulo = None
        
        if soup.title:
            titulo = soup.title.text.strip()
            if " – Wikipédia" in titulo:
                titulo = titulo.split(" – Wikipédia")[0].strip()
            elif " - Wikipédia" in titulo:
                titulo = titulo.split(" - Wikipédia")[0].strip()
            
        nome_arquivo = normalizar_nome_arquivo(titulo)
        caminho_arquivo = os.path.join(diretorio_dados, f"{nome_arquivo}.html")
        
        with open(caminho_arquivo, "wb") as f:
            f.write(conteudo)
            
        return nome_arquivo
    except Exception as e:
        print(f"Erro ao salvar página: {str(e)}")
        return None

print("Iniciando coleta de páginas da Wikipedia...")
while fila_paginas and total_paginas < max_paginas:
    url_atual = fila_paginas.pop(0)
    
    if url_atual in paginas_visitadas:
        continue
        
    print(f"Analisando página {total_paginas+1}/{max_paginas}: {url_atual}")
    
    paginas_visitadas.add(url_atual)
    
    conteudo = obter_pagina(url_atual)
    if not conteudo:
        continue
    
    soup = BeautifulSoup(conteudo, "html.parser")
    
    nome_arquivo = salvar_pagina(conteudo, soup)
    
    novos_links = processar_links(soup)
    
    fila_paginas.extend(novos_links)

    total_paginas += 1
    
    time.sleep(tempo_espera + random.random())

print(f"Coleta finalizada! Coletadas {total_paginas} páginas da Wikipedia.")

In [ ]:
if not os.path.exists(diretorio_dados):
    print(f"Erro: Pasta {diretorio_dados} não encontrada! Execute a Tarefa 1 primeiro.")

if not os.path.exists(diretorio_infoboxes):
    os.makedirs(diretorio_infoboxes)

def sanitizar_nome_arquivo(titulo):
    caracteres_invalidos = ['\\', '/', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in caracteres_invalidos:
        titulo = titulo.replace(char, ' ')
    
    while '  ' in titulo:
        titulo = titulo.replace('  ', ' ')
    
    titulo = titulo.strip()
    
    if len(titulo) > 100:
        titulo = titulo[:100]
    
    return titulo

def gerar_nome_arquivo_unico(titulo_base, diretorio):
    nome_arquivo = f"{titulo_base}.json"
    caminho_completo = os.path.join(diretorio, nome_arquivo)
    
    contador = 1
    while os.path.exists(caminho_completo):
        nome_arquivo = f"{titulo_base}_{contador}.json"
        caminho_completo = os.path.join(diretorio, nome_arquivo)
        contador += 1
        
    return nome_arquivo

def verificar_infobox(soup):
    for tabela in soup.find_all("table"):
        if not tabela.has_attr("class"):
            continue
            
        classes = tabela["class"] if isinstance(tabela["class"], list) else tabela["class"].split()
        
        if "infobox" in classes:
            return True, tabela
        elif "infobox_v2" in classes:
            return True, tabela
    
    return False, None

def extrair_texto(elemento):
    if not elemento:
        return ""
    
    for tag in elemento.find_all(["sup", "style", "script"]):
        tag.decompose()
    
    texto = elemento.get_text(separator=" ", strip=True)
    
    texto = texto.replace(" .", ".").replace(" ,", ",").replace(" :", ":").replace(" ;", ";")
    texto = texto.replace(" )", ")").replace("( ", "(").replace(" ]", "]").replace("[ ", "[")
    
    while "  " in texto:
        texto = texto.replace("  ", " ")
    
    return texto.strip()

def extrair_pares_chave_valor(infobox):
    dados_infobox = {}
    
    primeiro_th = infobox.find("th")
    
    for linha in infobox.find_all("tr"):
        celula_chave = linha.find("td", attrs={"scope": "row"})
        if celula_chave:
            chave = extrair_texto(celula_chave)
            if not chave:
                continue
                
            celulas_valor = linha.find_all("td")
            for i, td in enumerate(celulas_valor):
                if td == celula_chave and i+1 < len(celulas_valor):
                    valor = extrair_texto(celulas_valor[i+1])
                    dados_infobox[chave] = valor
                    break
        
        celula_chave = linha.find("th", attrs={"scope": "row"})
        if celula_chave:
            chave = extrair_texto(celula_chave)
            if not chave:
                continue
                
            celulas_valor = linha.find_all("td")
            if celulas_valor:
                valor = extrair_texto(celulas_valor[0])
                dados_infobox[chave] = valor
    
    for linha in infobox.find_all("tr"):
        ths = linha.find_all("th")
        tds = linha.find_all("td")
        
        if len(ths) == 1 and len(tds) == 1:
            if ths[0] != primeiro_th:
                chave = extrair_texto(ths[0])
                valor = extrair_texto(tds[0])
                
                if chave and not ths[0].has_attr("colspan"):
                    dados_infobox[chave] = valor
    
    return dados_infobox

def processar_pagina(caminho_arquivo):
    try:
        with open(caminho_arquivo, "r", encoding="utf-8") as f:
            conteudo = f.read()
        
        soup = BeautifulSoup(conteudo, "html.parser")
        
        tem_infobox, infobox = verificar_infobox(soup)
        if not tem_infobox:
            return None, None
        
        titulo = None
        
        primeiro_th = infobox.find("th")
        if primeiro_th:
            span_no_th = primeiro_th.find("span")
            if span_no_th:
                titulo = extrair_texto(span_no_th)
            else:
                titulo = extrair_texto(primeiro_th)
        
        dados_infobox = extrair_pares_chave_valor(infobox)
        
        return titulo, dados_infobox
    except Exception as e:
        print(f"Erro ao processar {caminho_arquivo}: {str(e)}")
        return None, None

arquivos_html = [os.path.join(diretorio_dados, arquivo) 
                for arquivo in os.listdir(diretorio_dados) 
                if arquivo.endswith(".html")]

total_arquivos = len(arquivos_html)
arquivos_com_infobox = 0
infoboxes_extraidas = 0

for i, caminho_arquivo in enumerate(arquivos_html):
    nome_arquivo = os.path.basename(caminho_arquivo)
    
    titulo, dados_infobox = processar_pagina(caminho_arquivo)
    
    if not titulo or not dados_infobox:
        continue
    
    if len(dados_infobox) == 0:
        continue
    
    arquivos_com_infobox += 1
    
    dados_finais = {"__arquivo_origem": nome_arquivo}
    dados_finais.update(dados_infobox)
    
    titulo_limpo = sanitizar_nome_arquivo(titulo)
    nome_json = gerar_nome_arquivo_unico(titulo_limpo, diretorio_infoboxes)
    caminho_json = os.path.join(diretorio_infoboxes, nome_json)
    
    with open(caminho_json, "w", encoding="utf-8") as f:
        json.dump(dados_finais, f, ensure_ascii=False, indent=4)
    
    infoboxes_extraidas += 1

print(f"\nConcluído! Encontradas {arquivos_com_infobox} páginas com infobox.")
print(f"Total de {infoboxes_extraidas} infoboxes extraídas com sucesso.")